In [ ]:
import requests

In [ ]:
pip install vk_api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 762.3 kB/s eta 0:00:00


In [ ]:
from vk_api import VkApi
from vk_api.utils import get_random_id
import requests
import json
import pandas as pd
import time
import numpy as np
import warnings

In [ ]:
VK_TOKEN = 'vk1.a.6XVezXP3LeQHZMCvWNSwn030-Eg1GEKf_aYXjwfgnpVX26s9f9sd7TPJ2-GnL9o_fhUzV3yb4NA0DMwdrlDFrD-RRTPATlJJDfF0KBkjpAAa7eHZ6fK4U3kRzGPpESuEF9-UKQAUg0Hqh7huZFWOuQG2R-kOYdE5XOTBeZd-aqYdodlEim00vyPxyo58L7Idy5OMeIIvFjLmg6W0PFivpQ'

In [ ]:
def get_wall_posts(VK_TOKEN: str) -> dict:
    #метод wall.get позволяет получать записи со стены
    url = 'https://api.vk.com/method/wall.get'
    params = {
        'access_token': VK_TOKEN, # ваш токен
        'owner_id': '-94', # id сообщества указывается со знаком - перед цифрами
        'v': '5.131', # версия API
        'offset': '0', #смещение (сколько постов надо пропустить, чтобы начать сбор)
        'count': 100 # сколько постов получить. максимум можно получить 100
    }

    response = requests.get(url, params=params)
    return response.json()


wall_posts = get_wall_posts(VK_TOKEN)
#print(wall_posts)

In [ ]:
data_posts = pd.DataFrame(columns = ['id_group', 'id_post', 'data', 'description', 'title', 'text', 'views', 'likes', 'reposts'])
#data_posts

собираем 100 постов одним запросом и записываем в датафрейм

In [ ]:
j = len(data_posts)

for i in range(len(wall_posts['response']['items'])):
    #описание и заголовок бывают у ссылок, видео, изображениий. если что то такое есть в посте, то это будет записано, если нет, то будет пустая строка
    try:
        link = wall_posts['response']['items'][i]['attachments'][0]['link']
        description = link['description']
        title = link['title']
    except:
        try:
            video = wall_posts['response']['items'][i]['attachments'][0]['video']
            description = video['description']
            title = video['title']
        except:
            try:
                photo = wall_posts['response']['items'][i]['attachments'][0]['photo']
                description = photo['description']
                title = photo['title']
            except:
                description, title = '',''

#если пост есть в датафрейме (проверяем по id поста), то перезаписываем в ту же строчку. кол-во лайков могло со временем измениться
    if wall_posts['response']['items'][i]['id'] in list(data_posts['id_post']):
        #записываем индекс этого поста в датафрейме
        index = data_posts[data_posts['id_post'] == wall_posts['response']['items'][i]['id']].index[0]
        #перезаписываем строку по найденному индексу
        data_posts.loc[index] = [wall_posts['response']['items'][i]['owner_id'],
                                wall_posts['response']['items'][i]['id'],
                                wall_posts['response']['items'][i]['date'],
                                description,
                                title,
                                wall_posts['response']['items'][i]['text'],
                                wall_posts['response']['items'][i]['views']['count'],
                                wall_posts['response']['items'][i]['likes']['count'],
                                wall_posts['response']['items'][i]['reposts']['count']
                                ]

    else:

        #записываем в датафрем необходимую информацию
        #id сообщества
        #id поста
        #дату поста
        #описание видео, ссылки, изображения если есть
        #заголовок видео, ссылки, изображения если есть
        #текст поста
        #кол-во просмотров поста
        #кол-во лайков
        #кол-во репостов
        data_posts.loc[j] = [wall_posts['response']['items'][i]['owner_id'],
                                wall_posts['response']['items'][i]['id'],
                                wall_posts['response']['items'][i]['date'],
                                description,
                                title,
                                wall_posts['response']['items'][i]['text'],
                                wall_posts['response']['items'][i]['views']['count'],
                                wall_posts['response']['items'][i]['likes']['count'],
                                wall_posts['response']['items'][i]['reposts']['count']
                                ]
        j += 1

In [ ]:
data_posts

,id_group,id_post,data,description,title,text,views,likes,reposts
0,-94,57196,1712926803,,,Пятница! А где тусуются айтишники? 👇,7394,11,6
1,-94,57194,1712912402,,,"Каждый из нас, своего рода, художник 👨🏻‍🎨 \n \...",1534,3,2
2,-94,57190,1712826001,,,"Мы его ждали и он возвращается! Грандиозный, л...",3624,89,55
3,-94,57182,1712752746,Где узнать больше о DevOps и IT в целом? 13 ап...,Клип @itmoru,Тут про свежую IT-конференцию👇🏻\n\nВ ролике Вя...,3701,18,11
4,-94,57181,1712749261,,,Прекрасные весенние новости! ИТМО — лучший рос...,4594,106,18
...,...,...,...,...,...,...,...,...,...
95,-94,56718,1700555854,,,"🃏 Зачем нам карты Таро, если есть искусственны...",10728,31,17
96,-94,56711,1700321497,"Мы уже начали поздравлять преподавателей, а вы...",Клип @itmoru,"Мы уже начали поздравлять преподавателей, а вы...",15391,193,23
97,-94,56703,1700229603,,,"🎄 Сммщик ждёт нг, чтобы загадать больше мемов ...",15747,43,4
98,-94,56702,1700217448,Готовимся поздравить преподавателей высшей шко...,Клип @itmoru,Готовимся поздравить преподавателей высшей шко...,18391,443,100


In [ ]:
wall_posts['response']['count']

15519

# автоматически получаем по 100 постов в каждом запросе, пока не соберем все посты сообщества

In [ ]:
data_posts = pd.DataFrame(columns = ['id_group', 'id_post', 'data', 'description', 'title', 'text', 'views', 'likes', 'reposts'])
#data_posts

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#получаем кол-во всех постов в сообществе
count_posts = wall_posts['response']['count']

c = 0
while c <= count_posts:
    print(c)
    #отправляем get запросы
    def get_wall_posts(VK_TOKEN: str) -> dict:
        #метод wall.get позволяет получать записи со стены
        url = 'https://api.vk.com/method/wall.get'
        params = {
            'access_token': VK_TOKEN, # ваш токен
            'owner_id': '-94', # id сообщества указывается со знаком - перед цифрами
            'v': '5.131', # версия API
            'offset': str(c), #смещение (сколько постов надо пропустить, чтобы начать сбор)
            'count': 100 # сколько постов получить. максимум можно получить 100
        }

        response = requests.get(url, params=params)
        return response.json()


    wall_posts = get_wall_posts(VK_TOKEN)


    #записываем полученные данные в датафрейм
    j = len(data_posts)

    for i in range(len(wall_posts['response']['items'])):
        #описание и заголовок бывают у ссылок, видео, изображениий. если что то такое есть в посте, то это будет записано, если нет, то будет пустая строка
        try:
            link = wall_posts['response']['items'][i]['attachments'][0]['link']
            description = link['description']
            title = link['title']
        except:
            try:
                video = wall_posts['response']['items'][i]['attachments'][0]['video']
                description = video['description']
                title = video['title']
            except:
                try:
                    photo = wall_posts['response']['items'][i]['attachments'][0]['photo']
                    description = photo['description']
                    title = photo['title']
                except:
                    description, title = '',''

    #если пост есть в датафрейме (проверяем по id поста), то перезаписываем в ту же строчку. кол-во лайков могло со временем измениться
        if wall_posts['response']['items'][i]['id'] in list(data_posts['id_post']):
            #записываем индекс этого поста в датафрейме
            index = data_posts[data_posts['id_post'] == wall_posts['response']['items'][i]['id']].index[0]
            #эта конструкция нужна для тех постов времен динозавров когда еще не отображалось кол-во просмотров поста
            try:
                views =  wall_posts['response']['items'][i]['views']['count']
            except:
                views = np.nan
            #перезаписываем строку по найденному индексу
            data_posts.loc[index] = [wall_posts['response']['items'][i]['owner_id'],
                                    wall_posts['response']['items'][i]['id'],
                                    wall_posts['response']['items'][i]['date'],
                                    description,
                                    title,
                                    wall_posts['response']['items'][i]['text'],
                                    views,
                                    wall_posts['response']['items'][i]['likes']['count'],
                                    wall_posts['response']['items'][i]['reposts']['count']
                                    ]

        else:

            #записываем в датафрем необходимую информацию
            #id сообщества
            #id поста
            #дату поста
            #описание видео, ссылки, изображения если есть
            #заголовок видео, ссылки, изображения если есть
            #текст поста
            #кол-во просмотров поста
            #кол-во лайков
            #кол-во репостов

            #эта конструкция нужна для тех постов времен динозавров когда еще не отображалось кол-во просмотров поста
            try:
                views =  wall_posts['response']['items'][i]['views']['count']
            except:
                views = np.nan
            data_posts.loc[j] = [wall_posts['response']['items'][i]['owner_id'],
                                    wall_posts['response']['items'][i]['id'],
                                    wall_posts['response']['items'][i]['date'],
                                    description,
                                    title,
                                    wall_posts['response']['items'][i]['text'],
                                    views,
                                    wall_posts['response']['items'][i]['likes']['count'],
                                    wall_posts['response']['items'][i]['reposts']['count']
                                    ]
            j += 1

            data_posts.to_csv('/content/drive/MyDrive/посты_сообщества_ИТМО.csv', index = False)

    c += 100
    #time.sleep(1)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500


In [67]:
data_posts

,id_group,id_post,data,description,title,text,views,likes,reposts
0,-94,57196,1712926803,,,Пятница! А где тусуются айтишники? 👇,7473.0,11,6
1,-94,57194,1712912402,,,"Каждый из нас, своего рода, художник 👨🏻‍🎨 \n \...",1542.0,3,2
2,-94,57190,1712826001,,,"Мы его ждали и он возвращается! Грандиозный, л...",3634.0,89,55
3,-94,57182,1712752746,Где узнать больше о DevOps и IT в целом? 13 ап...,Клип @itmoru,Тут про свежую IT-конференцию👇🏻\n\nВ ролике Вя...,3703.0,18,11
4,-94,57181,1712749261,,,Прекрасные весенние новости! ИТМО — лучший рос...,4597.0,106,18
...,...,...,...,...,...,...,...,...,...
15514,-94,5,1165260027,,,Всем привет!,NaN,0,0
15515,-94,4,1165259762,,,и почему тут никто не пишет?,NaN,1,0
15516,-94,3,1165258871,,,"ну вот, нас уже трое =)",NaN,0,0
15517,-94,2,1165257425,,,давай пытаться =),NaN,2,0


In [ ]:
data_posts.to_csv('/content/drive/MyDrive/посты_сообщества_ИТМО.csv', index = False)